In [107]:
import geopy
import requests
import pandas as pd
from datetime import datetime
from geopy.distance import geodesic
import logging
import pytz
import threading
import time
import sqlite3
path_extracao_onibus = 'extracaoPontos.ipynb'
def executar_script(script_path):
    if os.path.isfile(script_path):
        try:
            print(f"Executando {script_path}...")

            # Abrir o notebook
            with open(script_path) as f:
                nb = read(f, as_version=4)

            # Executar o notebook
            client = NotebookClient(nb)
            client.execute()

            print(f"{script_path} executado com sucesso.")
        except Exception as e:
            print(f"Erro ao executar {script_path}: {e}")
    else:
        print(f"O arquivo {script_path} não foi encontrado.")
# Função para criar a tabela no banco de dados SQLite (se ainda não existir)
def criar_tabela():
    with sqlite3.connect('dados_processados.db') as conn:
        conn.execute('''
            CREATE TABLE IF NOT EXISTS onibus (
                COD TEXT,
                REFRESH TEXT,
                LAT_IN_TIME REAL,
                LON_IN_TIME REAL,
                CODIGOLINHA TEXT,
                ADAPT INTEGER,
                TIPO_VEIC INTEGER,
                TABELA INTEGER,
                SITUACAO TEXT,
                SITUACAO2 TEXT,
                SENT TEXT,
                TCOUNT INTEGER,
                SENTIDO_IN_TIME TEXT,
                HORA TEXT,
                FLAG_PROCES INTEGER,
                PARADA_MAIS_PROXIMA TEXT,
                DISTANCIA_MINIMA REAL,
                SEQ INTEGER,
                SEQ_MAX INTEGER,
                ITINERARIO INTEGER
            )
        ''')
        conn.commit()
        
def carregar_tabela_para_dataframe(nome_tabela, banco_de_dados):
    conn = sqlite3.connect(banco_de_dados)
    
    try:
        # Verificar se a tabela existe
        cursor = conn.cursor()
        cursor.execute("""
            SELECT name FROM sqlite_master WHERE type='table' AND name=?;
        """, (nome_tabela,))
        
        resultado = cursor.fetchone()

        if resultado:
            # Se a tabela existe, carregar seus dados para um DataFrame
            df = pd.read_sql_query(f"SELECT * FROM {nome_tabela}", conn)
            return df
        else:
            # Retornar um DataFrame vazio se a tabela não existir
            return pd.DataFrame()  
    finally:
        conn.close()  # Garantir que a conexão seja fechada

def reseta_index(df):
    df = df.reset_index(drop=True)
    return df

def calcular_distancia(lat1, lon1, lat2, lon2):
    try:
        coords_1 = (lat1, lon1)
        coords_2 = (lat2, lon2)
        distancia = geopy.distance.geodesic(coords_1, coords_2).meters
        return distancia
    except Exception as e:
        logging.error("Erro ao calcular distancia: %s", e)
        return None
        
def retorna_itinerario(df_onibus, df_pontos):
    try:
        for index in range(df_onibus.shape[0] - 1):
            if df_onibus['COD'].iloc[index] == df_onibus['COD'].iloc[index + 1]:
                df_pontos_cod = df_pontos[df_pontos['COD'] == df_onibus['CODIGOLINHA'].iloc[0]]
                
                if not df_pontos_cod.empty:
                    itinerarios = df_pontos_cod['ID_do_Itinerario'].unique()

                    for itinerario_atual in itinerarios:
                        df_ponto_itinerario_atual = df_pontos_cod[
                            df_pontos_cod['ID_do_Itinerario'] == itinerario_atual
                        ]

                        for offset in [0, 1]:  # Processa o índice atual e o próximo
                            i = index + offset
                            try:
                                distancias = df_ponto_itinerario_atual.apply(
                                    lambda row: calcular_distancia(
                                        df_onibus['LAT_IN_TIME'].iloc[i], 
                                        df_onibus['LON_IN_TIME'].iloc[i], 
                                        row['Latitude'], 
                                        row['Longitude']
                                    ),
                                    axis=1
                                )

                                index_parada_mais_proxima = distancias.idxmin()
                                distancia_minima = distancias.min()
                                nome_parada_mais_proxima = df_ponto_itinerario_atual.loc[index_parada_mais_proxima, 'NOME']
                                mapeamento = dict(zip(df_ponto_itinerario_atual['NOME'], df_ponto_itinerario_atual['SEQ']))

                                # Atualiza as informações no DataFrame
                                df_onibus.at[i, 'PARADA_MAIS_PROXIMA'] = nome_parada_mais_proxima
                                df_onibus.at[i, 'SEQ'] = mapeamento[nome_parada_mais_proxima]
                                df_onibus.at[i, 'SEQ_MAX'] = df_ponto_itinerario_atual['SEQ'].max()
                                df_onibus.at[i, 'DISTANCIA_MINIMA'] = distancia_minima
                                # Verifica a sequência e retorna o itinerário se válido
                                if df_onibus['SEQ'].iloc[index] < df_onibus['SEQ'].iloc[index + 1]:
                                    return itinerario_atual  # Encerra o laço assim que achar um itinerário válido
                            except Exception as e:
                                logging.error("Erro ao processar distâncias: %s", e)
                                continue  # Continua para o próximo índice


                else:
                    # Se não achar os pontos, atualiza o banco de dados
                    print('atualizando os pontos')
                    executar_script(path_extracao_onibus)
    except Exception as e:
        logging.error("Erro ao retornar itinerário: %s", e)
        return None  # Retorna None em caso de erro

def atribuir_parada(df_onibus, df_pontos):
    try:
        # Percorre todo o DataFrame df_onibus
        for i in range(df_onibus.shape[0]):
            # Obter as coordenadas da linha de ônibus atual
            lat_onibus = df_onibus['LAT_IN_TIME'].iloc[i]
            lon_onibus = df_onibus['LON_IN_TIME'].iloc[i]
            codigo_atual = df_onibus['CODIGOLINHA'].iloc[i]  # Código da linha atual
            itinerario_atual = df_onibus['ITINERARIO'].iloc[i]  # Itinerário da linha atual
            if itinerario_atual != 0: 
                # Filtrar pontos do itinerário correspondente
                df_pontos_itinerario = df_pontos[df_pontos['ID_do_Itinerario'] == itinerario_atual].reset_index(drop=True)
    
                # Calcular distâncias para todas as paradas
                distancias = df_pontos_itinerario.apply(
                    lambda row: calcular_distancia(
                        lat_onibus,
                        lon_onibus,
                        row['Latitude'],
                        row['Longitude']
                    ),
                    axis=1
                )
    
                if not distancias.empty:  # Verifica se a série de distâncias não está vazia
                    index_parada_mais_proxima = distancias.idxmin()
                    distancia_minima = distancias.min()
                    nome_parada_mais_proxima = df_pontos_itinerario.loc[index_parada_mais_proxima, 'NOME']
                    mapeamento = dict(zip(df_pontos_itinerario['NOME'], df_pontos_itinerario['SEQ']))
    
                    # Atualiza as informações diretamente no DataFrame df_onibus
                    df_onibus.at[i, 'PARADA_MAIS_PROXIMA'] = nome_parada_mais_proxima
                    df_onibus.at[i, 'SEQ'] = mapeamento[nome_parada_mais_proxima]
                    df_onibus.at[i, 'SEQ_MAX'] = df_pontos_itinerario['SEQ'].max()
                    df_onibus.at[i, 'DISTANCIA_MINIMA'] = distancia_minima
                else:
                    pass
    except Exception as e:
        logging.error("Erro ao atribuir paradas: %s", e)

    return df_onibus  # Retorna o DataFrame atualizado

def retorna_df_itinerario(df_onibus, df_pontos):
    try:
        if df_pontos.empty:
            logging.warning("O DataFrame df_pontos está vazio.")
            return None
        if df_onibus.empty:
            logging.warning("O DataFrame df_onibus está vazio.")
            return None

        if 'PARADA_MAIS_PROXIMA' not in df_onibus.columns:
            df_onibus['PARADA_MAIS_PROXIMA'] = 'inicializado'
        if 'DISTANCIA_MINIMA' not in df_onibus.columns:
            df_onibus['DISTANCIA_MINIMA'] = 0.0
        if 'SEQ' not in df_onibus.columns:
            df_onibus['SEQ'] = -1
        if 'SEQ_MAX' not in df_onibus.columns:
            df_onibus['SEQ_MAX'] = -1
        if 'ITINERARIO' not in df_onibus.columns:
            df_onibus['ITINERARIO'] = 0
        
        codigos_linhas = df_onibus['CODIGOLINHA'].unique()
 
        for codigo in codigos_linhas:
            #selecionar o dataframe com as linhas do codigolinha
            df_linha_atual = df_onibus[df_onibus['CODIGOLINHA'] == codigo]
            df_linha_atual = df_linha_atual[df_linha_atual['SENT'] != '']
            df_linha_atual = reseta_index(df_linha_atual)
            
            #se não há valor circular ou vazio '' na coluna SENT 
            if df_linha_atual['SENT'].isin(['IDA', 'VOLTA']).all():
                #selecionar os dois sentidos -> IDA e VOLTA
                df_sent_ida = df_linha_atual[df_linha_atual['SENT'] == 'IDA']
                df_sent_ida = reseta_index(df_sent_ida)
                df_sent_volta = df_linha_atual[df_linha_atual['SENT'] == 'VOLTA']
                df_sent_volta = reseta_index(df_sent_volta)

                itinerario_ida = retorna_itinerario(df_sent_ida, df_pontos)
                itinerario_volta = retorna_itinerario(df_sent_volta, df_pontos)

           

                # Atualiza o DataFrame principal com os itinerários encontrados
                if itinerario_ida:
                    df_onibus.loc[
                        (df_onibus['CODIGOLINHA'] == codigo) & (df_onibus['SENT'] == 'IDA'), 'ITINERARIO'
                    ] = itinerario_ida

                if itinerario_volta:
                    df_onibus.loc[
                        (df_onibus['CODIGOLINHA'] == codigo) & (df_onibus['SENT'] == 'VOLTA'), 'ITINERARIO'
                    ] = itinerario_volta


        df_onibus_final = atribuir_parada(df_onibus, df_pontos)
        return df_onibus_final
    except Exception as e:
        logging.error("Erro ao retornar DataFrame do itinerário: %s", e)
        return None

# Criar a tabela se não existir
criar_tabela()

# Carregar os DataFrames
df_onibus = carregar_tabela_para_dataframe('onibus', 'dados_onibus.db')
df_pontos = carregar_tabela_para_dataframe('onibus', 'dados_pontos.db')

# Verificar se os DataFrames não estão vazios
if not df_onibus.empty and not df_pontos.empty:
    

    # Remover valores nulos
    df_onibus = df_onibus.dropna(subset=['LAT_IN_TIME', 'LON_IN_TIME'])
    df_pontos = df_pontos.dropna(subset=['Latitude', 'Longitude'])
    df = retorna_df_itinerario(df_onibus, df_pontos)

    # Salvar as alterações no banco
    salvar_df_no_banco(df_onibus, 'onibus', 'dados_processados.db')
else:
    print("Erro: Um ou ambos os DataFrames estão vazios.")